In [1]:
import os
import torch
import numpy as np
import pandas as pd
import wfdb
from PIL import Image
from matplotlib import pyplot as plt
from tqdm import tqdm
import seaborn as sns
from app.ecg.ecg import Datasets, EcgSignal
from sklearn.metrics import confusion_matrix, classification_report

ptbxl_data = pd.read_csv(r'C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\labels.csv', index_col=0)
# ptbxl_statements = pd.read_csv(Datasets.ptbxl_scp_statements.path, index_col=0)
# ptbxl_data['patient_id'] = ptbxl_data['patient_id'].astype(int)
# ptbxl_data['nurse'] = ptbxl_data['nurse'].astype('Int64')
# ptbxl_data['site'] = ptbxl_data['site'].astype('Int64')
# ptbxl_data['validated_by'] = ptbxl_data['validated_by'].astype('Int64')


In [4]:
ptbxl_data.info()

In [15]:

def get_file_paths(filename):
    filename_lr = filename.split('/')[-1].split('_')[0]
    return fr"C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\npy_signals100\{filename_lr}.npy"

clean_tags = lambda x: [e.replace("'", "")  for e in x[1:-1].split(', ')]
ptbxl_data["file_paths"] = ptbxl_data["filename_lr"].apply(get_file_paths)
ptbxl_data["superdiagnostic"] = ptbxl_data["superdiagnostic"].apply(clean_tags)

In [3]:
ptbxl_data

# Data preparation

In [16]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit(ptbxl_data["superdiagnostic"].values)
mlb.classes_.tolist()

In [17]:
train_labels = mlb.transform(ptbxl_data["superdiagnostic"].tolist())
ptbxl_data[mlb.classes_.tolist()] = train_labels

# ECGNet

In [ ]:
ecg_idx = pd.read_csv(r'C:\Users\redmi\PycharmProjects\ecg-tool-api\data\ptbxl\ptbxl_database.csv')
ptbxl_data['ecg_id']=ecg_idx['ecg_id']

In [18]:
from app.ecg.ecg import EcgDataset

valid_df = ptbxl_data[ptbxl_data['strat_fold'] == 9]
test_df = ptbxl_data[ptbxl_data['strat_fold'] == 10]

dataset = EcgDataset(ptbxl_data)
valid_dataset = EcgDataset(valid_df)
test_dataset = EcgDataset(test_df)


In [ ]:
import torch
from sklearn.metrics import roc_auc_score
from models.ecgnet import ECGNet
import matplotlib.pyplot as plt
import joblib
import numpy as np

model = ECGNet()
model = model.double()
model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\ecgnet_weights_authors.pt'
model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')))
model.eval()

In [ ]:
ecg, label = valid_dataset[151]
label

In [ ]:
prediction = model(ecg)
prediction

In [ ]:
softmaxed_pred = torch.softmax(prediction, dim=1).to(torch.float64)

In [ ]:
print(softmaxed_pred.detach().numpy(), label.numpy().astype(int))

In [ ]:
roc_auc_score(label.numpy().astype(int), prediction.detach().numpy().reshape(-1), average='macro')

In [ ]:
len(test_dataset)

In [ ]:
gt_all = []
pred_all = []

for id in tqdm(range(len(test_dataset))):
    ecg , label = test_dataset[id]
    # print(ecg, label)
    prediction = model(ecg)
    pred_all.append(prediction.detach().numpy())
    gt_all.append(label.detach().numpy().astype(int))
gt_all_array = np.vstack(gt_all)
pred_all_array = np.vstack(pred_all)


In [ ]:
gt_all_array

In [65]:
from models.metrics import Metrics, metric_summary, AUC, roc_auc_score


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def evaluate_model(model, loader):
    gt_all = []
    pred_all = []
    for id in tqdm(range(len(loader))):
        ecg , label = loader[id]
        # print(ecg, label)
        prediction = model(ecg)
        # pred_all.append(prediction.detach().numpy())
        pred_all.append(prediction.detach().numpy())
        # pred_all.append(sigmoid(prediction.detach().numpy()))
        gt_all.append(label.detach().numpy().astype(int))
    gt_all_array = np.vstack(gt_all)
    pred_all_array = np.vstack(pred_all)
    roc_score = roc_auc_score(gt_all_array, pred_all_array, average="macro")
    acc, mean_acc = Metrics(gt_all_array, pred_all_array)
    class_auc = AUC(gt_all_array, pred_all_array)
    summary = metric_summary(gt_all_array, pred_all_array)
    print(f"class wise accuracy: {acc}")
    print(f"accuracy: {mean_acc}")
    print(f"roc_score : {roc_score}")
    print(f"class wise AUC : {class_auc}")
    print(f"F1 score (Max): {summary[0]}")
    print(f"class wise precision, recall, f1 score : {summary}")
    return gt_all_array, pred_all_array
    

## EcgNet scores

In [ ]:


Metrics(gt_all_array, pred_all_array)

In [ ]:
print("ecgNet(ptbxlv1.0.2) - test - aurroc- macro: ", roc_auc_score(gt_all_array, pred_all_array))

In [ ]:
roc_score = roc_auc_score(gt_all_array, pred_all_array, average="macro")
acc, mean_acc = Metrics(gt_all_array, pred_all_array)
class_auc = AUC(gt_all_array, pred_all_array)
summary = metric_summary(gt_all_array, pred_all_array)

print(f"class wise accuracy: {acc}")
print(f"accuracy: {mean_acc}")
print(f"roc_score : {roc_score}")
print(f"class wise AUC : {class_auc}")
print(f"F1 score (Max): {summary[0]}")
print(f"class wise precision, recall, f1 score : {summary}")

In [ ]:
pred_all_array

In [ ]:
pred_all_array.shape

In [ ]:
print("ecgNet - valid - aurroc- macro: ", roc_auc_score(np.vstack(gt_all), np.vstack(pred_all)))

# ResNet50 CQT

In [31]:
from models.nn.custom_model_cqt import CustomModel
from models.config import CqtCFG
import torch

model_resnet50cqt = CustomModel(CqtCFG)
# model = model.double()
model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\resnet50\resnet50d_fold9_best_score.pth'
model_resnet50cqt.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cpu')), strict=False)
model_resnet50cqt.eval()


In [ ]:
valid_dataset = EcgDataset(valid_df, feature='cqt')
test_dataset = EcgDataset(test_df, feature='cqt')

In [ ]:
cqt_im, label = valid_dataset[0]
# cqt_im = cqt_im[None, None, :]
cqt_im.shape

In [ ]:
model_resnet50cqt(cqt_im)

In [33]:
gt_v, labels_v = evaluate_model(model_resnet50cqt, valid_dataset)

In [34]:
gt_t, labels_t = evaluate_model(model_resnet50cqt, test_dataset)

In [21]:
len(valid_dataset)

In [ ]:
from models.nn.xresnet1d import xresnet1d101
from models.nn.inception1d import inception1d
from models.nn.resnet1d import resnet1d_wang
from models.nn.rnn1d import RNN1d

def model_factory(model_name):
    model = None
    if model_name.lower()=='xresnet1d101':
        model = xresnet1d101(input_channels=12, num_classes=5)

    if model_name.lower()=='resnet1d_dwang':
        model = resnet1d_wang(input_channels=12, num_classes=5)

    if model_name.lower()=='inception1d_model':
        model = inception1d(input_channels=12, num_classes=5)

    if model_name.lower()=='rnn_1d':
        model = RNN1d(input_channels=12, num_classes=5)

    return model

# Xresnet1d

In [41]:
from models.nn.xresnet1d import xresnet1d101
import torch



xresnet1d_model = xresnet1d101(input_channels=12, num_classes=5)
xresnet1d_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\xresnet1d101\exp0_xresnet1d101xresnet1d101_fold1_19epoch_best_score.pth'
xresnet1d_model.load_state_dict(torch.load(xresnet1d_model_weights_path, map_location=torch.device('cpu'))['model'])
xresnet1d_model.double()
xresnet1d_model.eval()

In [45]:
torch.load(xresnet1d_model_weights_path, map_location=torch.device('cpu'))['model']

In [8]:
ecg, label = valid_dataset[7]
ecg.shape

In [52]:
label

In [53]:
xresnet1d_model(ecg)

In [34]:
test_dataset[107]

In [54]:
evaluate_model(xresnet1d_model, test_dataset)

In [55]:
evaluate_model(xresnet1d_model, valid_dataset)

In [42]:
evaluate_model(xresnet1d_model, valid_dataset)

# ResNET1d_wang

In [43]:
from models.nn.resnet1d import resnet1d_wang
import torch

resnet1d_wang_model = resnet1d_wang(input_channels=12, num_classes=5)
resnet1d_wang_weights = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\resnet1d_wang\resnet1d_wang_fold1_16epoch_best_score.pth'
# xresnet1d_model.load_state_dict(torch.load(xresnet1d_model_weights_path, map_location=torch.device('cpu'))['model'])

resnet1d_wang_model.load_state_dict(torch.load(resnet1d_wang_weights, map_location=torch.device('cpu'))['model'])
resnet1d_wang_model.double()
resnet1d_wang_model.eval()



In [11]:
resnet1d_wang_model(ecg)

In [12]:
evaluate_model(resnet1d_wang_model, test_dataset)

In [13]:
evaluate_model(resnet1d_wang_model, valid_dataset)

In [44]:
evaluate_model(resnet1d_wang_model, test_dataset)

# Inception1d

In [82]:
from models.nn.inception1d import inception1d

inception1d_model = inception1d(input_channels=12, num_classes=5)
inception1d_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\inception1d\inception1d_fold1_15epoch_best_score.pth'
inception1d_model.load_state_dict(torch.load(inception1d_model_weights_path, map_location=torch.device('cpu'))['model'])
inception1d_model.double()
inception1d_model.eval()

In [58]:
ecg, label

In [59]:
inception1d_model(ecg)

In [60]:
evaluate_model(inception1d_model, test_dataset)

In [62]:
e, l = test_dataset[107]
l

In [68]:
torch.softmax(inception1d_model(e), dim=1).detach().numpy()

In [81]:
plt.plot(e[0,6].detach().numpy())

In [75]:
e[0,1]

# RNN1d

In [17]:
from models.nn.rnn1d import RNN1d

rnn1d_model = RNN1d(input_channels=12, num_classes=5)
rnn1d_model.double()
rnn1d_model.eval()



# Metrics

In [14]:
gt, scores = evaluate_model(xresnet1d_model, test_dataset)
# cm, y_pred = calculate_metrics(scores)

In [60]:
from sklearn.metrics import multilabel_confusion_matrix
from models.metrics import MR, multilabel_accuracy, zero_one_loss, multilabel_recall, multilabel_precision, multilabel_hamming_loss
from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, classification_report


def sigmoid(x):
    return 1 / (1 + np.exp(-x))


def calculate_metrics(scores):
    
    y_pred=[]
    for sample in  tqdm(scores):
      sample = sigmoid(sample)
      y_pred.append([1 if i>=0.5 else 0 for i in sample ] )
    y_pred = np.array(y_pred)
    
    confusion_matrix = multilabel_confusion_matrix(gt, y_pred)
    print("Confusion matrix: \n", confusion_matrix)
    
    match_ratio = MR(gt, y_pred)
    print("\nMR: ", match_ratio.round(3))
    
    mlb_accuracy = multilabel_accuracy(gt, y_pred)
    print("\nAccuracy: ", mlb_accuracy.round(3))

    zoneloss = zero_one_loss(gt, y_pred)
    print("\n0/1Loss: ", zoneloss.round(3))
    
    mlb_recall = multilabel_recall(gt, y_pred)
    print("\nRecall: ", mlb_recall.round(3))
    
    mlb_precision = multilabel_precision(gt, y_pred)
    print("\nPrecision: ", mlb_precision.round(3))
    
    mlb_hamming_loss = multilabel_hamming_loss(gt, y_pred)
    print("\nHamming loss: ", mlb_hamming_loss)

    return confusion_matrix, y_pred
   

   


In [84]:
cm, y_pred = calculate_metrics(scores)

In [36]:
cm[0]

In [80]:
## import matplotlib.pyplot as plt
from sklearn.metrics import multilabel_confusion_matrix
import numpy as np



# Определяем названия классов
classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']

# Визуализация матриц ошибок в одну строку
fig, axes = plt.subplots(1, 5, figsize=(20, 4))

for i, (matrix, class_name) in enumerate(zip(cm, classes)):
    axes[i].matshow(matrix, cmap="Pastel2_r")
    axes[i].set_title(class_name)
    for (j, k), value in np.ndenumerate(matrix):
        axes[i].text(k, j, f"{value}", ha='center', va='center')
    axes[i].set_xlabel("Predicted label")
    axes[i].set_ylabel("True label")

plt.tight_layout()
plt.show()


In [45]:
plot_confusion_matrix(cm, ['CD', 'HYP', 'MI', 'NORM', 'STTC'])

In [43]:
title_size = 16
plt.rcParams.update({'font.size':16})
display_labels = ['CD', 'HYP', 'MI', 'NORM', 'STTC']  
colorbar = False
cmap = "Blues"  # Try "Greens". Change the color of the confusion matrix.
## Please see other alternatives at https://matplotlib.org/stable/tutorials/colors/colormaps.html
values_format = ".3f"  # Determine the number of decimal places to be displayed.

# Create subplots for given confusion matrices
f, axes = plt.subplots(1, 5, figsize=(10, 16))

# Plot the first confusion matrix (Model 1) at position (0, 0)
axes[0, 0].set_title("Model 1", size=title_size)
ConfusionMatrixDisplay(confusion_matrix=cm[0], display_labels=display_labels).plot(
    include_values=True, cmap=cmap, ax=axes[0, 1], colorbar=colorbar, values_format=values_format)

# # Remove x-axis labels and ticks
# axes[0, 0].xaxis.set_ticklabels(['', '', '', ''])
# axes[0, 0].set_xlabel('')
# axes[0, 0].tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)


# # Plot the second confusion matrix (Model 2) at position (0, 1)
# axes[0, 1].set_title("Model 2", size=title_size)
# ConfusionMatrixDisplay(confusion_matrix=cm01, display_labels=display_labels).plot(
#     include_values=True, cmap=cmap, ax=axes[0, 1], colorbar=colorbar, values_format=values_format)

In [1]:
gt

In [92]:
classification_report(gt, y_pred)

# VGG 16

In [76]:
from torch_ecg.utils.utils_nn import adjust_cnn_filter_lengths
from torch_ecg.model_configs import ECG_CRNN_CONFIG
from torch_ecg.models.ecg_crnn import ECG_CRNN


# xresnet1d_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\xresnet1d101\exp0_xresnet1d101xresnet1d101_fold1_19epoch_best_score.pth'
# xresnet1d_model.load_state_dict(torch.load(xresnet1d_model_weights_path, map_location=torch.device('cpu'))['model'])
# xresnet1d_model.double()
# xresnet1d_model.eval()

vgg16_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\vgg16\vgg16_fold1_10epoch_best_score.pth'
config = adjust_cnn_filter_lengths(ECG_CRNN_CONFIG, fs=100)
config.cnn.name="vgg16"
classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']
n_leads = 12
vgg16_model = ECG_CRNN(classes, n_leads, config)
vgg16_model.load_state_dict(torch.load(vgg16_model_weights_path, map_location=torch.device('cpu'))['model'])
vgg16_model.double()
vgg16_model.eval()

In [25]:
gt, scores = evaluate_model(vgg16_model, test_dataset)

In [66]:
gt, scores = evaluate_model(vgg16_model, test_dataset)

In [67]:
cm, y_pred = calculate_metrics(scores)

In [72]:
print(classification_report(gt, y_pred, target_names=classes))

# MobileNet


In [27]:
mobilenet_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\mobilenet\mobilenet_v3_fold1_3epoch_best_score.pth'
config = adjust_cnn_filter_lengths(ECG_CRNN_CONFIG, fs=100)
config.cnn.name="mobilenet_v3"
classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']
n_leads = 12
mobilenet = ECG_CRNN(classes, n_leads, config)
mobilenet.load_state_dict(torch.load(mobilenet_model_weights_path, map_location=torch.device('cpu'))['model'])
mobilenet.double()
mobilenet.eval()

In [29]:
gt, scores = evaluate_model(mobilenet, test_dataset)

# RegNet

In [30]:
# C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\regnet\regnet_27_24_fold1_10epoch_best_score.pth
regnet_model_weights_path = r'C:\Users\redmi\PycharmProjects\ecg-tool-api\models\pretrained\regnet\regnet_27_24_fold1_10epoch_best_score.pth'
config = adjust_cnn_filter_lengths(ECG_CRNN_CONFIG, fs=100)
config.cnn.name="regnet_27_24"
classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']
n_leads = 12
regnet = ECG_CRNN(classes, n_leads, config)
regnet.load_state_dict(torch.load(regnet_model_weights_path, map_location=torch.device('cpu'))['model'])
regnet.double()
regnet.eval()

In [31]:
gt, scores = evaluate_model(regnet, test_dataset)

In [38]:
gt, scores = evaluate_model(regnet, test_dataset)

In [39]:
scores

In [75]:
config

In [107]:
import torchvision
from torchview import draw_graph
# import

model_graph = draw_graph(inception1d(input_channels=12, num_classes=5), input_size=( 1, 12, 1000), expand_nested=True)
model_graph.visual_graph

In [110]:
config = adjust_cnn_filter_lengths(ECG_CRNN_CONFIG, fs=100)
config.cnn.name="vgg16"
classes = ['CD', 'HYP', 'MI', 'NORM', 'STTC']
n_leads = 12
# vgg16_model = ECG_CRNN(classes, n_leads, config)



model_graph = draw_graph(ECG_CRNN(classes, n_leads, config), input_size=(1, 12, 1000), expand_nested=True)
model_graph.visual_graph

In [9]:
class SimpleRNN(torch.nn.Module):
    """Simple RNN"""

    def __init__(self, inplace: bool = True) -> None:
        super().__init__()
        self.hid_dim = 2
        self.input_dim = 3
        self.max_length = 4
        self.lstm = torch.nn.LSTMCell(self.input_dim, self.hid_dim)
        self.activation = torch.nn.LeakyReLU(inplace=inplace)
        self.projection = torch.nn.Linear(self.hid_dim, self.input_dim)

    def forward(self, token_embedding: torch.Tensor) -> torch.Tensor:
        b_size = token_embedding.size()[0]
        hx = torch.randn(b_size, self.hid_dim, device=token_embedding.device)
        cx = torch.randn(b_size, self.hid_dim, device=token_embedding.device)

        for _ in range(self.max_length):
            hx, cx = self.lstm(token_embedding, (hx, cx))
            hx = self.activation(hx)

        return hx

In [97]:
model_graph_1 = draw_graph(
    SimpleRNN(), input_size=(2, 3),
    graph_name='RecursiveNet',
    roll=True
)

In [100]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz-11.0.0-win64/bin/'
model_graph_1.visual_graph

In [11]:
## import hiddenlayer as hl
import torchvision
import torch
from torchview import draw_graph
import os

os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz-11.0.0-win64/bin/'

# AlexNet
model = torchvision.models.alexnet()

# Build HiddenLayer graph
hl_graph = hl.build_graph(SimpleRNN(), torch.zeros([2, 3])).build_dot()

# Use a different color theme
hl_graph.theme = hl.graph.THEMES["blue"].copy()  # Two options: basic and blue
hl_graph

In [113]:
model